```{currentmodule} optimap
```

In [ ]:
# Code snippet for rendering animations in the docs
from IPython.display import HTML
import warnings
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

def render_ani_func(f):
    om.utils.disable_interactive_backend_switching()
    plt.switch_backend('Agg')
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ani = f()
    %matplotlib inline
    om.utils.enable_interactive_backend_switching()

    vid = HTML(ani.to_html5_video())
    plt.close('all')
    return vid

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/apd.ipynb>`, or a {download}`python script <converted/apd.py>` with code cells. We highly recommend using [Visual Studio Code](#vscode) to execute this tutorial.
```

# Tutorial 8: Action Potential Duration

This tutorial discusses how to compute action potential durations (APDs) and calcium transient durations (CaTDs) from cardiac optical mapping data using ``optimap``. In addition to computing upstroke timing, repolarization timing for different wave heights, we will also compute APD histograms, APD maps, CaTD histograms and CaTD maps.

```{warning}
This tutorial is currently work in progress. We will add more information soon.
```

First, we will load a voltage- and calcium-sensitive optical mapping recording and split the video into the two channels:

In [ ]:
import optimap as om

filename = om.utils.retrieve_example_data('optimap-example-file-03.npy')
video_voltage = om.load_video(filename, start_frame=0, step=2)
video_calcium = om.load_video(filename, start_frame=1, step=2)

om.print_properties(video_voltage)
om.print_properties(video_calcium)

You can plot optical traces interactively as follows:

In [ ]:
traces, positions = om.select_traces(video_voltage, size=3, fps=1000)

or define positions and then display the corresponding optical traces manually:

In [ ]:
positions = [(51, 61), (55, 53)]

traces = om.extract_traces(video_voltage, positions, size=3, show=True, fps=1000)

In this recording, the action potentials exhibit the notch during the depolarization phase. The corresponding calcium transients look as follows:

In [ ]:
traces = om.extract_traces(video_calcium, positions, size=3, show=True, fps=1000)